# Sentence Transformer

In [33]:
from sentence_transformers import SentenceTransformer, util
import torch
import json

In [34]:
torch.cuda.is_available()

True

In [35]:
embedder = SentenceTransformer('gtr-t5-xxl', device='cuda')


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.73G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
data = json.load(open('dataNew.json', 'r'))

In [ ]:
corpus = list()
hrefList = list()
for i in data:
    corpus.extend([data[i]['text']])
    hrefList.extend([data[i]['link']])

In [ ]:
len(corpus)

18

In [ ]:
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [ ]:
top_k = min(5, len(corpus))

In [ ]:
query = "I have a requirement where i have to include validation logic in flow service if the request is coming from a certain client. The client is hitting the API gateway URL and from there request is routed to ESB service. I need to set a value in my gateway so that when the call reaches ESB i would know whether to include logic/not. I have configured an input variable(authReq) in ESB service, how to pass the authReq variable with a value (Yes/No) from API gateway to webMethods flow service?"


In [ ]:
query_embedding = embedder.encode(query, convert_to_tensor=True)

In [32]:
cos_scores = util.dot_score(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)
print("Query:", query)
print("Top 5 most similar sentences in corpus:")
for score, idx in zip(top_results[0], top_results[1]):
    print(corpus[idx], "(Score: {:.4f}) | href:{}".format(score, hrefList[idx]))

Query: I have a requirement where i have to include validation logic in flow service if the request is coming from a certain client. The client is hitting the API gateway URL and from there request is routed to ESB service. I need to set a value in my gateway so that when the call reaches ESB i would know whether to include logic/not. I have configured an input variable(authReq) in ESB service, how to pass the authReq variable with a value (Yes/No) from API gateway to webMethods flow service?
Top 5 most similar sentences in corpus:
Overview of Security Configuration in API GatewayYou must have the API Gateway's manage security configurations functional privilege assigned to perform the following tasks in the security configuration section of API Gateway:Configure the keystores and truststores required for incoming and outgoing message-level and transport-level security.Configure ports of API Gateway.Configure the SAML issuer to use in API Gateway outbound authentication to fetch the SA

In [12]:
len(hrefList)

18

In [13]:
semanticSearchResult = util.semantic_search(
    query_embedding, corpus_embeddings)
for i in semanticSearchResult[0]:
    # print(i)
    print(corpus[i['corpus_id']],
          "(Score: {:.4f}) | href:{}".format(i['score'], hrefList[i['corpus_id']]))


How Do I Create an API by Importing from a File?To create an API by importing a file1. In the Create API page, select Import API from file. 2. Click Browse and select the file to import. 3. Provide the API name in the Name field, if required. 4. Provide the required Description, API Type, Version, and click Create. The API is created and appears in the APIs list under the APIs tab. (Score: 0.5413) | href:https://documentation.softwareag.com/webmethods/api_gateway/yai10-15/webhelp/yai-webhelp/yai-webhelp/ta-qsg_create_api_from_a_file.html#
How do I export or download runtime requests?This use case starts when you want to export the client request from your API Gateway instance to your local machine and ends when you import the request in another API Gateway instance.To export the runtime request1.  Click the Tracer tab. The Trace API page displays the Runtime events, Policies applied, and Event tracer details sections.2.  In the Runtime events section, select the runtime event that you 

In [14]:
corpus[semanticSearchResult[0][0]['corpus_id']]

'How Do I Create an API by Importing from a File?To create an API by importing a file1. In the Create API page, select Import API from file. 2. Click Browse and select the file to import. 3. Provide the API name in the Name field, if required. 4. Provide the required Description, API Type, Version, and click Create. The API is created and appears in the APIs list under the APIs tab.'

# FAISS

In [15]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, util
import json
import time

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
data = json.load(open('data.json', 'r'))
corpus = list()
hrefList = list()
for i in data:
    corpus.extend(data[i]['data'])
    hrefList.extend([data[i]['href']]*len(data[i]['data']))

In [17]:
faissModel = SentenceTransformer(
    'distilbert-base-nli-mean-tokens', device='cuda')
faissEmbedding = faissModel.encode(corpus, show_progress_bar=True)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/652 [00:00<?, ?it/s]

In [18]:
index = faiss.IndexHNSWFlat(faissEmbedding.shape[1], 32)
index = faiss.IndexIVFPQ(index, faissEmbedding.shape[1], 10000, 16, 8)
index.train(faissEmbedding)
index.add(faissEmbedding)

WARNING clustering 20843 points to 10000 centroids: please provide at least 390000 training points


In [19]:
faiss.write_index(index, 'faissHNSWFlat_IVFPQ.index')
faiss.read_index('faissHNSWFlat_IVFPQ.index')

<faiss.swigfaiss.IndexIVFPQ; proxy of <Swig Object of type 'faiss::IndexIVFPQ *' at 0x7fb622c966f0> >

In [20]:
def search(query):
    t=time.time()
    query_vector = faissModel.encode([query], show_progress_bar=True)
    k = 5
    top_k = index.search(query_vector, k)
    print('totaltime: {}'.format(time.time()-t))
    return [hrefList[_id] for _id in top_k[1].tolist()[0]]

In [21]:
# query=str(input())
results=search(query)
print('results :')
for result in results:
    print('\t',result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

totaltime: 0.023241281509399414
results :
	 https://documentation.softwareag.com/webmethods/api_gateway/yai10-11/10-11_API_Gateway_webhelp/api-gateway-integrated-webhelp/updating_apis_file.html#
	 https://documentation.softwareag.com/webmethods/mywebmethods_server/mws10-11/10-11_MWSw/my-webmethods-server-webhelp/ta-allowing_users_to_access_the_workspace_management_page.html#
	 https://documentation.softwareag.com/webmethods/mywebmethods_server/mws10-11/10-11_MWSw/my-webmethods-server-webhelp/to-system_2.html#
	 https://documentation.softwareag.com/webmethods/mywebmethods_server/mws10-11/10-11_MWSw/my-webmethods-server-webhelp/to-system_2.html#
	 https://documentation.softwareag.com/webmethods/mywebmethods_server/mws10-11/10-11_MWSw/my-webmethods-server-webhelp/to-system_2.html#
